In [14]:
import json
import pyperclip
import json
import re

# # 从文件读取 JSON
# with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_0.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# data = data[36]
# print(data["set_id"])
# # 1. 获取对话文本
# full_text = data["full_context"]

# # 2. 获取角色名称（是一个字典），并把它的所有 value（角色名）提取到列表
# characters_dict = data["characters"]
# speakers = list(characters_dict.values())  # ["Jamey", "Pearl", "Marithza", "Donyae"]

# 3. 定义一个函数，用于在角色名之前插入换行符
def bold_unbolded_speakers(text, speaker_list):
    for speaker in speaker_list:
        # 匹配不是加粗的 Name:，避免重复包裹 **
        # 要求前面不是 *，后面是冒号，但后面不能紧跟 **
        pattern = re.compile(rf'(?<!\*)\b({re.escape(speaker)}):(?!=\*)', re.IGNORECASE)
        text = pattern.sub(r'**\1:**', text)
    return text
def insert_line_breaks_bold_speakers(text, speaker_list):
    speaker_pattern = '|'.join(re.escape(s) for s in speaker_list)
    # 匹配所有的 **Speaker:** // **Speaker**:，无论前面是什么，直接加换行
    pattern = re.compile(rf'(\*\*(?:{speaker_pattern})(?::)?\*\*)')
    # 每个匹配前都加两个换行
    return pattern.sub(r'\n\n\1', text)

def remove_extra_blank_lines(text):
    # 把 3 个及以上连续换行缩减为 2 个
    return re.sub(r'\n{3,}', '\n\n', text)

# 定义一个正则，匹配常见的 emoji 区段
emoji_pattern = re.compile(
    "["  
    u"\U0001F600-\U0001F64F"  # Emoticons (表情)
    u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & Map symbols
    u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
    "]+",
    flags=re.UNICODE
)

def remove_emojis(text: str) -> str:
    """
    从文本中移除所有匹配到的 emoji。
    """
    return emoji_pattern.sub(' ', text)

def process_text(text, speakers):
    fixed_text = remove_emojis(text)
    fixed_text = bold_unbolded_speakers(fixed_text, speakers)
    fixed_text = insert_line_breaks_bold_speakers(fixed_text, speakers)
    fixed_text = remove_extra_blank_lines(fixed_text)
    return fixed_text.strip()

# with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_0.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# speakers = list(characters_dict.values())  # e.g. ["Jamey", "Pearl", "Marithza", "Donyae"]

# fixed_text = remove_emojis(full_text)
# fixed_text = bold_unbolded_speakers(fixed_text, speakers)  # 👈 先加粗所有没加的说话人名
# fixed_text = insert_line_breaks_bold_speakers(fixed_text, speakers)  # 👈 然后插入换行
# fixed_text = remove_extra_blank_lines(fixed_text)
# fixed_text = fixed_text.strip()

# print(fixed_text)
# pyperclip.copy(fixed_text)

In [28]:
# 主流程：读取原 JSON 文件，遍历所有对话样本，然后对每个样本中的 full_context 进行格式化处理

# 输入与输出文件路径
input_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/second_test/Tactful_conv_set_4.json"
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/second_test/Tactful_conv_set_4.json"

with open(input_filepath, "r", encoding="utf-8") as fin:
    all_data = json.load(fin)

new_data = []
for sample in all_data:
    # 处理 full_context
    full_text = sample.get("full_context", "")
    characters_dict = sample.get("characters", {})
    speakers = list(characters_dict.values())
    sample["full_context"] = process_text(full_text, speakers)
    
    # 如果存在 short_context，也进行同样的处理
    short_text = sample.get("short_context", "")
    sample["short_context"] = process_text(short_text, speakers)

    new_data.append(sample)

with open(output_filepath, "w", encoding="utf-8") as fout:
    json.dump(new_data, fout, ensure_ascii=False, indent=4)

print("处理完毕，新的 JSON 文件保存在：", output_filepath)

处理完毕，新的 JSON 文件保存在： /Users/liuyiwei/python/tactful-tom/dataset/second_test/Tactful_conv_set_4.json


In [29]:
# 打印一下查看是否修改好了格式
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/second_test/Tactful_conv_set_4.json"

with open(output_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

if data:
    sample = data[0]
    print("full_context:")
    print(sample["full_context"])
    # print("\nshort_context:")
    # print(sample.get("short_context", "N/A"))
else:
    print("没有找到数据。")

full_context:
**Osvaldo:** So, with Thanksgiving finally over, does anyone have plans for this weekend?

**Arjun:** I'm debating between just relaxing at home or going out for a movie. What about you guys?

**Cora:** There's a new art exhibit opening on Saturday downtown that I've been wanting to check out. Anyone interested in joining?

**Charles:** That sounds intriguing, Cora. I might want to tag along. Been a while since I've immersed myself in some art.

**Osvaldo:** Art exhibit sounds cool! I was hoping to catch a football game on TV, but it shouldn't take up the whole day. Could be a good mix.

**Arjun:** Maybe we could do both. Art in the afternoon, then a movie or the game in the evening?

**Cora:** That's a solid plan! We can even grab some food in between. I'm always up for trying new restaurants.

**Charles:** There's a cozy little café near the gallery I read about. Maybe we can stop by there for lunch after the exhibit.

**Osvaldo:** Perfect. I love the idea of having a m